In [ ]:
# Cell 1: Setup
import sys
import os
from pathlib import Path
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add scripts directory to path (adjust if structure differs)
repo_root = Path('.').resolve() # Assumes notebook runs from repo root
scripts_path = repo_root / 'scripts'
if str(scripts_path) not in sys.path:
    sys.path.append(str(scripts_path))

# Import project classes
from utils.config_loader import load_config
from inventory import Inventory
from ingest_inventory import InventoryIngestor # Assuming class structure now
from utils.api_clients import create_llm_client # For ingestion

# Load Config and Initialize
try:
    config_obj = load_config() # Searches for config.yaml
    inventory = Inventory(repo_root / "inventory.yaml")
    inventory.load()
    llm_client_inv = create_llm_client(config_obj) # LLM needed for ingestion
    ingestor = InventoryIngestor(inventory, llm_client_inv, repo_root) # Pass repo_root or project context
    print("Setup complete. Inventory loaded.")
except Exception as e:
    print(f"Error during setup: {e}")
    # Handle error appropriately, maybe stop execution


In [ ]:

# Cell 2: Display Inventory
inventory_df = inventory.to_dataframe()
display(inventory_df)


In [ ]:

# Cell 3: Ingest New Part from Image
upload_widget = widgets.FileUpload( accept='image/*', multiple=False, description="Upload Part Image")
ingest_button = widgets.Button(description="Ingest Part from Image")
output_area = widgets.Output()

def on_ingest_click(b):
    with output_area:
        output_area.clear_output()
        if not upload_widget.value:
            print("Please upload an image first.")
            return
        uploaded_file = upload_widget.value[0] # Using ipywidgets structure
        image_data = uploaded_file['content']
        try:
            print("Starting ingestion process (requires interactive input below)...")
            # Pass necessary dependencies if needed (e.g., lib_manager for footprint check during ingest)
            # ingestor = InventoryIngestor(inventory, llm_client_inv, project_context) # Adjust as needed
            success = ingestor.ingest_from_image_interactive(image_data) # Function handles prompts and saving
            if success:
                print("\nIngestion successful. Reloading inventory display...")
                inventory.load() # Reload data
                display(inventory.to_dataframe()) # Refresh display (or clear/redisplay cell 2)
            else:
                print("\nIngestion cancelled or failed.")
        except Exception as e:
            print(f"\nError during ingestion: {e}")
        finally:
             # Reset the file upload widget value so the same file can be uploaded again if needed
             upload_widget.value = ()


ingest_button.on_click(on_ingest_click)
display(upload_widget, ingest_button, output_area)


In [ ]:

# Cell 4: (Optional) Verify Inventory Footprints
# Add code here to call check_assets.py in a mode that only verifies inventory items
# Example:
# verifier = VerificationEngine(...) # Needs full setup
# inventory_report = verifier.verify_inventory(inventory)
# display(inventory_report)

# Cell 5: Save Inventory (if manual edits were done to DataFrame - complex)
# Generally recommend editing inventory via ingestion or directly in inventory.yaml
# save_button = widgets.Button(description="Save Inventory Changes")
# def on_save_click(b):
#     # Code to update inventory object from DataFrame edits (tricky)
#     # inventory.save()
#     print("Inventory Saved.")
# save_button.on_click(on_save_click)
# display(save_button)